In [1]:
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import backend as K
import os
import cv2
import glob
import PIL
import shutil
import numpy as np
import pandas as pd
from Arch_ResidualAttentionUnet import *
from metrics import *
from Datagenerators import *

from tensorflow import keras
import keras.backend as K
from keras.callbacks import CSVLogger
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.layers.experimental import preprocessing

np.set_printoptions(precision=3, suppress=True)

CSVLogger.on_test_begin = CSVLogger.on_train_begin
CSVLogger.on_test_batch_end = CSVLogger.on_epoch_end
CSVLogger.on_test_end = CSVLogger.on_train_end

In [2]:
model = build_resatt_unet(n_channels=3, ker_init='he_normal', dropout=0.2)
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics = ['accuracy',tf.keras.metrics.MeanIoU(num_classes=4), dice_coef, dice_coef_necrotic, dice_coef_edema ,dice_coef_enhancing] )

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 128, 128, 16  2320        ['conv2d[0][0]']                 
                                )                                                             

                                                                                                  
 add_5 (Add)                    (None, 8, 8, 128)    0           ['conv2d_transpose[0][0]',       
                                                                  'conv2d_16[0][0]']              
                                                                                                  
 activation_1 (Activation)      (None, 8, 8, 128)    0           ['add_5[0][0]']                  
                                                                                                  
 conv2d_18 (Conv2D)             (None, 8, 8, 1)      129         ['activation_1[0][0]']           
                                                                                                  
 activation_2 (Activation)      (None, 8, 8, 1)      0           ['conv2d_18[0][0]']              
                                                                                                  
 up_sampli

 add_8 (Add)                    (None, 32, 32, 64)   0           ['conv2d_30[0][0]',              
                                                                  'dropout_6[0][0]']              
                                                                                                  
 conv2d_31 (Conv2D)             (None, 32, 32, 32)   2080        ['add_8[0][0]']                  
                                                                                                  
 activation_6 (Activation)      (None, 32, 32, 32)   0           ['conv2d_31[0][0]']              
                                                                                                  
 conv2d_33 (Conv2D)             (None, 32, 32, 32)   1056        ['activation_6[0][0]']           
                                                                                                  
 conv2d_transpose_2 (Conv2DTran  (None, 32, 32, 32)  9248        ['conv2d_33[0][0]']              
 spose)   

                                )                                                                 
                                                                                                  
 batch_normalization_3 (BatchNo  (None, 128, 128, 16  64         ['conv2d_43[0][0]']              
 rmalization)                   )                                                                 
                                                                                                  
 concatenate_3 (Concatenate)    (None, 128, 128, 48  0           ['up_sampling2d_7[0][0]',        
                                )                                 'batch_normalization_3[0][0]']  
                                                                                                  
 conv2d_44 (Conv2D)             (None, 128, 128, 16  6928        ['concatenate_3[0][0]']          
                                )                                                                 
          

In [3]:
directory = [f.path for f in os.scandir(PATH) if f.is_dir()]


def generate_ids(dirLst):
    ids = []
    for i in range(0,len(dirLst)):
        ids.append(dirLst[i][dirLst[i].rfind('/')+1:])
    return ids

train_and_test_ids = generate_ids(directory); 

    
train_test_ids, val_ids = train_test_split(train_and_test_ids, train_size=236, test_size=60, random_state=14) 
train_ids, test_ids = train_test_split(train_test_ids, test_size=36, random_state=14) 

In [4]:
training_generator = DataGenerator2D(train_ids)
valid_generator = DataGenerator2D(val_ids)
test_generator = DataGenerator2D(test_ids)

In [5]:
csv_logger = CSVLogger('Logs/training/resattU_net_2d.log', separator=',', append=False)

eval_csv_logger = CSVLogger('Logs/testing/resattU_net_2d.log', separator=',', append=False)


callbacks = [
      keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.000001, verbose=1),
        csv_logger
    ]

In [6]:
K.clear_session()

history =  model.fit(training_generator,
                    epochs=100,
                    steps_per_epoch=len(train_ids),
                    callbacks= callbacks,
                    validation_data = valid_generator
                    )  
model.save("Models/2D/resattU_net_2d.h5")

Epoch 1/100
200/200 [==============================] - 69s 305ms/step - loss: 0.1210 - accuracy: 0.9786 - mean_io_u: 0.6954 - dice_coef: 0.3262 - dice_coef_necrotic: 0.1257 - dice_coef_edema: 0.3049 - dice_coef_enhancing: 0.1456 - val_loss: 0.0629 - val_accuracy: 0.9824 - val_mean_io_u: 0.6119 - val_dice_coef: 0.3080 - val_dice_coef_necrotic: 0.0704 - val_dice_coef_edema: 0.2489 - val_dice_coef_enhancing: 0.1764 - lr: 0.0010
Epoch 2/100
200/200 [==============================] - 60s 299ms/step - loss: 0.0386 - accuracy: 0.9886 - mean_io_u: 0.7090 - dice_coef: 0.4163 - dice_coef_necrotic: 0.2399 - dice_coef_edema: 0.4856 - dice_coef_enhancing: 0.3726 - val_loss: 0.0702 - val_accuracy: 0.9822 - val_mean_io_u: 0.7825 - val_dice_coef: 0.3781 - val_dice_coef_necrotic: 0.1731 - val_dice_coef_edema: 0.3176 - val_dice_coef_enhancing: 0.3578 - lr: 0.0010
Epoch 3/100
200/200 [==============================] - ETA: 0s - loss: 0.0305 - accuracy: 0.9904 - mean_io_u: 0.7134 - dice_coef: 0.4676 - dic

Epoch 17/100
200/200 [==============================] - 55s 273ms/step - loss: 0.0210 - accuracy: 0.9930 - mean_io_u: 0.7497 - dice_coef: 0.5492 - dice_coef_necrotic: 0.4221 - dice_coef_edema: 0.6722 - dice_coef_enhancing: 0.6585 - val_loss: 0.0970 - val_accuracy: 0.9749 - val_mean_io_u: 0.8006 - val_dice_coef: 0.4439 - val_dice_coef_necrotic: 0.2848 - val_dice_coef_edema: 0.4290 - val_dice_coef_enhancing: 0.3968 - lr: 1.0000e-06
Epoch 18/100
200/200 [==============================] - 55s 274ms/step - loss: 0.0210 - accuracy: 0.9930 - mean_io_u: 0.7498 - dice_coef: 0.5496 - dice_coef_necrotic: 0.4238 - dice_coef_edema: 0.6727 - dice_coef_enhancing: 0.6596 - val_loss: 0.1011 - val_accuracy: 0.9732 - val_mean_io_u: 0.7984 - val_dice_coef: 0.4430 - val_dice_coef_necrotic: 0.2909 - val_dice_coef_edema: 0.4295 - val_dice_coef_enhancing: 0.3959 - lr: 1.0000e-06
Epoch 19/100
200/200 [==============================] - 55s 273ms/step - loss: 0.0210 - accuracy: 0.9930 - mean_io_u: 0.7499 - dice_

Epoch 36/100
200/200 [==============================] - 55s 274ms/step - loss: 0.0208 - accuracy: 0.9930 - mean_io_u: 0.7490 - dice_coef: 0.5526 - dice_coef_necrotic: 0.4273 - dice_coef_edema: 0.6760 - dice_coef_enhancing: 0.6615 - val_loss: 0.1007 - val_accuracy: 0.9737 - val_mean_io_u: 0.8000 - val_dice_coef: 0.4444 - val_dice_coef_necrotic: 0.2899 - val_dice_coef_edema: 0.4289 - val_dice_coef_enhancing: 0.3962 - lr: 1.0000e-06
Epoch 37/100
200/200 [==============================] - 55s 274ms/step - loss: 0.0208 - accuracy: 0.9931 - mean_io_u: 0.7492 - dice_coef: 0.5531 - dice_coef_necrotic: 0.4276 - dice_coef_edema: 0.6765 - dice_coef_enhancing: 0.6626 - val_loss: 0.0964 - val_accuracy: 0.9754 - val_mean_io_u: 0.8018 - val_dice_coef: 0.4461 - val_dice_coef_necrotic: 0.2853 - val_dice_coef_edema: 0.4289 - val_dice_coef_enhancing: 0.3967 - lr: 1.0000e-06
Epoch 38/100
200/200 [==============================] - 55s 274ms/step - loss: 0.0208 - accuracy: 0.9931 - mean_io_u: 0.7490 - dice_

Epoch 55/100
200/200 [==============================] - 55s 273ms/step - loss: 0.0205 - accuracy: 0.9931 - mean_io_u: 0.7506 - dice_coef: 0.5563 - dice_coef_necrotic: 0.4315 - dice_coef_edema: 0.6808 - dice_coef_enhancing: 0.6657 - val_loss: 0.1002 - val_accuracy: 0.9743 - val_mean_io_u: 0.8031 - val_dice_coef: 0.4461 - val_dice_coef_necrotic: 0.2869 - val_dice_coef_edema: 0.4294 - val_dice_coef_enhancing: 0.3962 - lr: 1.0000e-06
Epoch 56/100
200/200 [==============================] - 55s 273ms/step - loss: 0.0205 - accuracy: 0.9931 - mean_io_u: 0.7499 - dice_coef: 0.5565 - dice_coef_necrotic: 0.4328 - dice_coef_edema: 0.6807 - dice_coef_enhancing: 0.6664 - val_loss: 0.0984 - val_accuracy: 0.9749 - val_mean_io_u: 0.8026 - val_dice_coef: 0.4467 - val_dice_coef_necrotic: 0.2867 - val_dice_coef_edema: 0.4288 - val_dice_coef_enhancing: 0.3972 - lr: 1.0000e-06
Epoch 57/100
200/200 [==============================] - 55s 274ms/step - loss: 0.0205 - accuracy: 0.9931 - mean_io_u: 0.7501 - dice_

Epoch 74/100
200/200 [==============================] - 80s 401ms/step - loss: 0.0202 - accuracy: 0.9932 - mean_io_u: 0.7513 - dice_coef: 0.5599 - dice_coef_necrotic: 0.4363 - dice_coef_edema: 0.6849 - dice_coef_enhancing: 0.6698 - val_loss: 0.1021 - val_accuracy: 0.9736 - val_mean_io_u: 0.8032 - val_dice_coef: 0.4463 - val_dice_coef_necrotic: 0.2886 - val_dice_coef_edema: 0.4270 - val_dice_coef_enhancing: 0.3980 - lr: 1.0000e-06
Epoch 75/100
200/200 [==============================] - 58s 289ms/step - loss: 0.0202 - accuracy: 0.9932 - mean_io_u: 0.7514 - dice_coef: 0.5602 - dice_coef_necrotic: 0.4374 - dice_coef_edema: 0.6850 - dice_coef_enhancing: 0.6708 - val_loss: 0.1042 - val_accuracy: 0.9728 - val_mean_io_u: 0.8017 - val_dice_coef: 0.4457 - val_dice_coef_necrotic: 0.2912 - val_dice_coef_edema: 0.4243 - val_dice_coef_enhancing: 0.3986 - lr: 1.0000e-06
Epoch 76/100
200/200 [==============================] - 56s 278ms/step - loss: 0.0202 - accuracy: 0.9932 - mean_io_u: 0.7513 - dice_

Epoch 93/100
200/200 [==============================] - 55s 274ms/step - loss: 0.0200 - accuracy: 0.9933 - mean_io_u: 0.7538 - dice_coef: 0.5636 - dice_coef_necrotic: 0.4421 - dice_coef_edema: 0.6895 - dice_coef_enhancing: 0.6731 - val_loss: 0.1015 - val_accuracy: 0.9739 - val_mean_io_u: 0.8043 - val_dice_coef: 0.4479 - val_dice_coef_necrotic: 0.2893 - val_dice_coef_edema: 0.4257 - val_dice_coef_enhancing: 0.3995 - lr: 1.0000e-06
Epoch 94/100
200/200 [==============================] - 55s 274ms/step - loss: 0.0200 - accuracy: 0.9933 - mean_io_u: 0.7541 - dice_coef: 0.5637 - dice_coef_necrotic: 0.4425 - dice_coef_edema: 0.6890 - dice_coef_enhancing: 0.6749 - val_loss: 0.1066 - val_accuracy: 0.9722 - val_mean_io_u: 0.8031 - val_dice_coef: 0.4469 - val_dice_coef_necrotic: 0.2931 - val_dice_coef_edema: 0.4274 - val_dice_coef_enhancing: 0.3971 - lr: 1.0000e-06
Epoch 95/100
200/200 [==============================] - 55s 274ms/step - loss: 0.0200 - accuracy: 0.9933 - mean_io_u: 0.7535 - dice_

In [ ]:
# model = keras.models.load_model('Models/U_net_200_2d_3ch.h5', 
#                                    custom_objects={ 'accuracy' : tf.keras.metrics.MeanIoU(num_classes=4),
#                                                    "dice_coef": dice_coef,
#                                                    "dice_coef_necrotic": dice_coef_necrotic,
#                                                    "dice_coef_edema": dice_coef_edema,
#                                                    "dice_coef_enhancing": dice_coef_enhancing
#                                                   }, compile=True)

In [7]:
model.evaluate(test_generator, callbacks=eval_csv_logger, verbose=1)

36/36 [==============================] - 10s 288ms/step - loss: 0.1394 - accuracy: 0.9668 - mean_io_u: 0.7976 - dice_coef: 0.4651 - dice_coef_necrotic: 0.3303 - dice_coef_edema: 0.4389 - dice_coef_enhancing: 0.4852


[0.13943137228488922,
 0.9667786955833435,
 0.7976285219192505,
 0.465082585811615,
 0.3303024172782898,
 0.4389362633228302,
 0.4852178990840912]

In [8]:
valid_csv_logger = CSVLogger('Logs/validation/resattU_net_2d.log', separator=',', append=False)
model.evaluate(valid_generator, callbacks=valid_csv_logger, verbose=1)

60/60 [==============================] - 12s 195ms/step - loss: 0.1040 - accuracy: 0.9735 - mean_io_u: 0.8058 - dice_coef: 0.4474 - dice_coef_necrotic: 0.2850 - dice_coef_edema: 0.4281 - dice_coef_enhancing: 0.3970


[0.10396437346935272,
 0.9734541177749634,
 0.8058194518089294,
 0.4474223852157593,
 0.2850046455860138,
 0.4280940592288971,
 0.39704594016075134]